In [2]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
%matplotlib inline

## This notebook will base on the following question:

Each day, your friend Jack has one of three moods: happy, neutral, or unhappy. If he’s happy, there is a 20% chance of being unhappy the next day, and an equal chance of him being either happy or neutral the next day. If he’s neutral, there is a 60% chance of staying neutral, and 30% chance of becoming happy. If he’s unhappy, there is a 50% chance he’ll still be unhappy the next day, and a 30% chance of becoming neutral.

Jack is away for the term, but you are in contact with him every day through a messaging app. You’ve noticed that if he is happy, he will reply to your message that day with 90% probability. If he is neutral, he has a 70% chance of replying, and if he is unhappy he will only reply with 20% probability.

Thefollowingisthesequenceofreplies(R)andnoreplies(N)overtheprevious 25 days:

$$[R,R,R,N,N,R,R,R,R,R,N,R,R,N,R,R,R,R,N,R,N,N,N,N,N]$$

We will find

1. The probability that Jack replies within 3 days using __forward recursion__.

2. The most possible moods of Jack for the past 25 days using __viterbi algorithm__

## 1.1 Model Visualization

<img src="Model.png">


## 1.2 Forward Recursion




For the forward recursion, we will use the following recurrence relationship:

\begin{align*}
	\alpha(z_{n}) &= p(x_{1:n},z_{n}) = p(x_{n}\mid z_{n}) \sum_{z_{n-1}}\alpha(z_{n-1})p(z_{n}\mid z_{n-1})&& \text{ for } n=2,\cdots ,T\\
	\alpha(z_{1}) &= p(x_{1},z_1)=p(z_{1})p(x_{1}\mid z_{1})
\end{align*}

Where $z_{i}$ are hidden states and $x_{i}$ are visible states. Using the forward recursion, we should be able to get $p(z_{n},x_{1:n})$ and in order to get the posterior distribution of $z_{n}$, all we need to do is to use the Bayes' Theorem:

\begin{align*}
	p(z_{n}\mid x_{1:n}) = \frac{p(z_{n},x_{1:n})}{p(x_{1:n})} = \frac{\alpha(z_{n})}{\sum_{z_{n}}\alpha(z_{n})}
\end{align*}

## 1.2.1 Implementation


In [3]:
def ForwardRecursion(z1, X, A, B):
    """
    Inputs :
        z1 :  An array sum to 1 representing the initial hidden states
        X  :  An array of int of length N representing the observations 
        A  :  The transition matrix, whose row sum to 1
        B  :  The emission matrix, whose row sum to 1
    Outputs:
        res:  The posterior for hidden state N
    """
    # number of steps
    N = len(X)
    
    # initialize p(x1, z1)
    alpha = z1 * B.T[X[0]]
    
    for i in range(1, N):
        alpha = B.T[X[i]] * np.dot(A.T, alpha)
        
    return alpha/np.sum(alpha)

## 1.2.2 Calculate hidden states

In [24]:
# Value initialization

# we will assume the initial hidden states are uniform
z1 = np.ones(3)/3
A = np.array([[0.4, 0.4, 0.2], [0.3, 0.6, 0.1], [0.2, 0.3, 0.5]])
B = np.array([[0.9, 0.1], [0.7, 0.3], [0.2, 0.8]])

R = 0; N = 1 # since we have reply as column 1 and not reply as column 2
X = np.array([R,R,R,N,N,R,R,R,R,R,N,R,R,N,R,R,R,R,N,R,N,N,N,N,N])

## 1.2.3 Calculate visible states

In [25]:
z = ForwardRecursion(z1, X, A, B) # posterior distribution for *today's* hidden state
print(f"Hidden states for today is {z}")

Hidden states for today is [0.05420019 0.26415651 0.6816433 ]


In [26]:
z_tmr = np.dot(A.T, z)
x_tmr = np.dot(B.T, z_tmr)
print(f"Hidden states for tomorrow is {z_tmr}")
print(f"Probability Jack will reply tomorrow is {x_tmr[0]}")

Hidden states for tomorrow is [0.23725569 0.38466697 0.37807734]
Probability Jack will reply tomorrow is 0.5584124679825492


## 1.2.4 More Calculation

For first question, we need to calculate $P(R \text{ within three days})=1-P(x_{t+1}=N, x_{t+2}=N, x_{t+3}=N)$, where $P(x_{t})$ represents the visible states for today and $P(x_{t+1})$ for tomorrow etc. For sake of simplicity, let $\bar{x} \equiv (x=N)$ and we have

\begin{align*}
	P(x_{t+1}=N, x_{t+2}=N, x_{t+3}=N) &= \sum_{z_{t},z_{t+1},z_{t+2},z_{t+3}}P(\bar x_{t+1}, \bar x_{t+2}, \bar x_{t+3}, z_{t},z_{t+1},z_{t+2})\\
	&= \sum_{z_{t:t+3}}P(\bar x_{t+3}\mid z_{t+3})P(z_{t+3}\mid z_{t+2})P(\bar x_{t+2}\mid z_{t+2})P(z_{t+2}\mid z_{t+1})P(\bar x_{t+1}\mid z_{t+1})P(z_{t+1}\mid z_{t})P(z_{t})\\
	&= \sum_{z_{t+3}} P(\bar x_{t+3}\mid z_{t+3}) \sum_{z_{t+2}} P(z_{t+3}\mid z_{t+2})P(\bar x_{t+2}\mid z_{t+2}) \sum_{z_{t+1}}P(z_{t+2}\mid z_{t+1})P(\bar x_{t+1}\mid z_{t+1})\sum_{z_{t}}P(z_{t+1}\mid z_{t})P(z_{t})\\
	&= \sum B_{N}^{T}\odot \Big( A^{T} B_{N}^{T}\odot \big(A^{T}B_{N}^{T}\odot (A^{T}P(z_{t}))\big) \Big)
\end{align*}

Where $P(z_{t})$ is the mood distribution for today, $B_{N}\in \mathbb{R}^{3}$ is the probability of not replying given all three moods and $A\in \mathbb R^{3\times 3}$ is the transition matrix, whose row sum to 1. 

We could actually summary this using a for loop. Let $h_{t+1} = B_{N}^{T}\odot (A^{T}h_{t})$ with initial value $h_{0} = P(z_{t})$ and we can say that $P(x_{t+1}=N, x_{t+2}=N, x_{t+3}=N)=\sum h_{3}$.

In [27]:
h = z

for _ in range(3):
    h = B.T[N] * (A.T @ h)

no_reply = sum(h)
print(f"The probablity that Jack replies in next three days is {1-no_reply}")

The probablity that Jack replies in next three days is 0.9135623744495275


## 1.3 Viterbi Algorithm

Recall that for the Viterbi Algorithm, we have


\begin{align*}
	\omega_{k}\left (z_{k}\right ) &= \max_{z_{1:k-1}} p(z_{1:k}, x_{1:k})\\
	&= \max_{z_{1:k-1}} p(x_{k}\mid z_{k}) p(z_{k}\mid z_{k-1}) p(z_{1:k-1},x_{1:k-1})\\
	&= \max_{z_{k-1}} p(x_{k}\mid z_{k}) p(z_{k}\mid z_{k-1}) \omega_{k-1}(z_{k-1})\\
\end{align*}

With initial condition:
\begin{align*}
	\omega_{1}(z_{1}) &= p(z_{1})p(x_{1}\mid z_{1})
\end{align*}

In [29]:
def viterbi(X, A, B, z1):    
    T = len(X) # num of obeservations
    M = len(A) # num of states
    
    # NOTE: we will use log scale to avoid underflow
    omega = np.log(z1 * B.T[X[0]]) # init cond
    # we will use prev to back track the hidden states
    prev = []
    
    for i in range(1, T):
        state = [0] * M
        for j in range(M):
            alpha = omega + np.log(A.T[j]) + np.log(B[j, X[i]])
            state[j] = np.argmax(alpha)
            omega[j] = np.max(alpha)
        prev.append(state)
    
    print(prev)
    path = [] # the path has reverse order
    path.append(np.argmax(omega)) # append the last state
    for i in range(T-2, -1, -1):
        path.append(prev[i][path[-1]]) # backtrack
    
    return path[::-1]

In [30]:
viterbi(X, A, B, z1)

[[0, 1, 2], [0, 1, 0], [0, 1, 2], [1, 1, 2], [1, 1, 2], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 1, 2], [1, 1, 2], [1, 1, 2]]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]